In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv, SAGEConv, GATConv, ChebConv
from torch_geometric.data import Data, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# GNN Model Definition
class My_GNN_GNN_NN(nn.Module):
    def __init__(self, node_size, feat_in, feat_size1, feat_size2, hidden_size1, output_size, gnn_type='GCN'):
        super(My_GNN_GNN_NN, self).__init__()
        self.gnn_type = gnn_type
        
        # Select GNN Layer Type
        if gnn_type == 'GCN':
            self.conv1 = GCNConv(feat_in, feat_size1)
            self.conv2 = GCNConv(feat_size1, feat_size2)
        elif gnn_type == 'GraphConv':
            self.conv1 = GraphConv(feat_in, feat_size1)
            self.conv2 = GraphConv(feat_size1, feat_size2)
        elif gnn_type == 'SAGEConv':
            self.conv1 = SAGEConv(feat_in, feat_size1)
            self.conv2 = SAGEConv(feat_size1, feat_size2)
        elif gnn_type == 'GATConv':
            self.conv1 = GATConv(feat_in, feat_size1)
            self.conv2 = GATConv(feat_size1, feat_size2)
        elif gnn_type == 'ChebConv':
            self.conv1 = ChebConv(feat_in, feat_size1, K=2)
            self.conv2 = ChebConv(feat_size1, feat_size2, K=2)
        else:
            raise ValueError("Invalid GNN type. Choose from 'GCN', 'GraphConv', 'SAGEConv', 'GATConv', 'ChebConv'.")

        self.lin1 = nn.Linear(node_size * feat_size2, hidden_size1)
        self.lin2 = nn.Linear(hidden_size1, output_size)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = x.flatten(start_dim=0)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

# Utility Functions
def normalize_dataset(x, y):
    x_mean, x_std = torch.mean(x, 0), torch.std(x, 0)
    y_mean, y_std = torch.mean(y, 0), torch.std(y, 0)
    x_std[x_std == 0] = 1
    y_std[y_std == 0] = 1
    x_norm = (x - x_mean) / x_std
    y_norm = (y - y_mean) / y_std
    return x_norm, y_norm, x_mean, y_mean, x_std, y_std

def denormalize_output(y_norm, y_mean, y_std):
    return y_norm * y_std + y_mean

def MSE(yhat, y):
    return torch.mean((yhat - y) ** 2)

# Training and Evaluation
def run_experiment(dataset, n_bus, gnn_type):
    feat_in = 7
    feat_size1, feat_size2 = 16, 8
    hidden_size1 = 64
    output_size = n_bus * 2
    lr = 0.0001
    patience = 50

    # Initialize model, optimizer, scheduler
    model = My_GNN_GNN_NN(n_bus, feat_in, feat_size1, feat_size2, hidden_size1, output_size, gnn_type).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience)

    # Training Loop
    best_val_loss = float('inf')
    train_loss_list, val_loss_list = [], []

    for epoch in range(600):
        model.train()
        train_loss = 0.0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            y_pred = model(batch).view(-1)
            loss = MSE(denormalize_output(y_pred, y_val_mean, y_val_std), denormalize_output(batch.y.view(-1), y_val_mean, y_val_std))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader.dataset)
        train_loss_list.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                y_val_pred = model(batch).view(-1)
                loss = MSE(denormalize_output(y_val_pred, y_val_mean, y_val_std), denormalize_output(batch.y.view(-1), y_val_mean, y_val_std))
                val_loss += loss.item()

        val_loss /= len(val_loader.dataset)
        val_loss_list.append(val_loss)

        # Scheduler and Early Stopping
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"Best_{gnn_type}_{n_bus}_bus_model.pt")

        # Logging
        if epoch % 10 == 0:
            print(f"Epoch {epoch} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    # Plot Loss Curve
    plt.plot(train_loss_list, label='Train Loss')
    plt.plot(val_loss_list, label='Validation Loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.title(f"{gnn_type} on {n_bus}-bus System")
    plt.show()

# Run experiments for all configurations
datasets = [(14, '14_bus_data.xlsx'), (30, '30_bus_data.xlsx'), (57, '57_bus_data.xlsx')]
gnn_types = ['GCN', 'GraphConv', 'SAGEConv', 'GATConv', 'ChebConv']

for n_bus, dataset_file in datasets:
    for gnn_type in gnn_types:
        print(f"Running {gnn_type} on {n_bus}-bus dataset...")
        run_experiment(dataset_file, n_bus, gnn_type)
